In [71]:
from collections import Counter
import numpy as np
import pandas as pd

data = {
    'AB': {'next_paths': ['BC', 'BD'], 'distance': 100, 'ucs': 80},
    'AC': {'next_paths': ['CD', 'CE'], 'distance': 80, 'ucs': 83},
    'BC': {'next_paths': ['CD', 'CE'], 'distance': 155, 'ucs': 83},
    'BD': {'next_paths': ['DE', 'DF'], 'distance': 125, 'ucs': 185},
    'CD': {'next_paths': ['DE', 'DF'], 'distance': 95, 'ucs': 185},
    'CE': {'next_paths': ['EF', 'EG'], 'distance': 125, 'ucs': 185},
    'DE': {'next_paths': ['EF', 'EG'], 'distance': 130, 'ucs': 185},
    'DF': {'next_paths': ['FG', 'FH'], 'distance': 120, 'ucs': 70},
    'EF': {'next_paths': ['FG', 'FH'], 'distance': 195, 'ucs': 70},
    'EG': {'next_paths': ['GI'], 'distance': 130, 'ucs': 50},
    'FG': {'next_paths': ['GI'], 'distance': 175, 'ucs': 50},
    'FH': {'next_paths': ['HG', 'HI'], 'distance': 120, 'ucs': 60},
    'GI': {'next_paths': [], 'distance': 110, 'ucs': 60},
    'HG': {'next_paths': ['GI'], 'distance': 150, 'ucs': 50},
    'HI': {'next_paths': [], 'distance': 80, 'ucs': 70}
}

# Convert data dictionary into a DataFrame
df_init = pd.DataFrame.from_dict(data, orient='index').reset_index()
df_init.columns = ['Possible Path', 'Next Possible Path', 'Distance', 'UCS']

# add Pheromone column in df, it will be updated as the algorithm runs
df_init['Pheromone'] = 1

In [72]:
paths = []
for i in df_init.index:
    path_init = df_init['Possible Path'].iloc[i]
    new_var = path_init.lower()    
    new_var = df_init.iloc[i].values.tolist()
    paths.append(df_init.iloc[i].values.tolist())

# Calculate sum of distances and UCS values for all paths
sum_dist = sum(path[2] for path in paths)
sum_ucs = sum(path[3] for path in paths)

In [73]:
# Calculate sum of distances and UCS values for all paths
sum_dist = sum(path[2] for path in paths)
sum_ucs = sum(path[3] for path in paths)

# Calculate length for multiple objectives (distance and UCS) with weights
def length(ant, dist_weight = 0.2, ucs_weight = 0.8):
    summ = 0
    for i in ant:
        for j in paths:
            if i == j[0]:
                sums = (dist_weight * j[2] / sum_dist) + (ucs_weight * j[3] / sum_ucs)
                summ += sums
    return summ

In [74]:

# probabiity calculation for each path with weighted fitness
def probabs(adja, dist_weight = 0.2, ucs_weight = 0.8):
    pher_path_quality = []
    for i in adja:
        for j in paths:
            if j[0] == i:
                # Calculate pheromone and path quality for "distance"
                dis_pher_pq = j[4] * (1 / j[2])
                dis_pher_pq = dis_pher_pq / sum_dist

                # Calculate pheromone and path quality for "ucs"
                ucs_pher_pq = j[4] * (1 / j[3])
                ucs_pher_pq = ucs_pher_pq / sum_ucs

                pher_pq = (dist_weight * dis_pher_pq) + (ucs_weight * ucs_pher_pq)
                pher_path_quality.append(pher_pq)

    # Calculate probability for each path
    summ = sum(pher_path_quality)
    probs = [pher_pq / summ for pher_pq in pher_path_quality]

    return probs

In [75]:
# Running the algorithm with weighted fitness
def full_aco(iter_number, ant_number, init_path_1, init_path_2, dist_weight = 0.2, ucs_weight = 0.8, evap_constant = 0.3):

    for i in range(iter_number):
        evaporation(constant=evap_constant)
        ants = []
        for j in range(ant_number):
            path = ant(init_path_1, init_path_2)
            ants.append(path)
        update_pherom(ants)
    return paths, ants

In [76]:
# initiate the ant to pass the random path
def ant(init_path_1, init_path_2):
    path = []

    starter = choose_paths([init_path_1[0], init_path_2[0]])
    path.append(starter)

    for i in paths:
        if path[-1] == i[0]:
            adj = i[1]
            if len(adj)==0:
                break
            else:
                adj_random = choose_paths(adj)
                path.append(adj_random)
    return path

In [77]:
# pheromone evaporation
def evaporation(constant):
    for i in paths:
        i[4] = i[4] * (1 - constant)

# pheromone update
def update_pherom(ants):
    for i in ants:
        pherom = 1 / (length(i))
        for j in i:
            for k in paths:
                if k[0] == j:
                    k[4] = k[4] + pherom

In [78]:
# randomly choosing path based on probability roulette
def choose_paths(adja):
    probab = probabs(adja)
    thresholds = np.cumsum(probab)
    r = np.random.random()

    for i, threshold in enumerate(thresholds):
        if r < threshold:
            return adja[i]

    return adja[-1]

# Initial paths for ants
init_path_1 = ['AB']
init_path_2 = ['AC']

# Running the algorithm with weighted fitness
paths, ants = full_aco(iter_number = 1000, ant_number = 500, init_path_1 = init_path_1, init_path_2 = init_path_2, dist_weight = 0.2, ucs_weight = 0.8)


In [79]:
print('\nChoosen path after running the full algorithm with weighted fitness')
print(Counter([str(i) for i in ants]).most_common())


Choosen path after running the full algorithm with weighted fitness
[("['AC', 'CE', 'EG', 'GI']", 500)]


In [80]:
print('\nAll Paths after running the full algorithm with weighted fitness')
for i in paths:
    print(i)


All Paths after running the full algorithm with weighted fitness
['AB', ['BC', 'BD'], 100, 80, 5.936454185405557e-143]
['AC', ['CD', 'CE'], 80, 83, 6578.111422913933]
['BC', ['CD', 'CE'], 155, 83, 5.936453353683898e-143]
['BD', ['DE', 'DF'], 125, 185, 8.317229244179944e-150]
['CD', ['DE', 'DF'], 95, 185, 6.15681386335847e-132]
['CE', ['EF', 'EG'], 125, 185, 6578.111422913933]
['DE', ['EF', 'EG'], 130, 185, 2.7096526908214486e-151]
['DF', ['FG', 'FH'], 120, 70, 6.15681386335847e-132]
['EF', ['FG', 'FH'], 195, 70, 6.281988994028764e-149]
['EG', ['GI'], 130, 50, 6578.111422913933]
['FG', ['GI'], 175, 50, 6.15681386335505e-132]
['FH', ['HG', 'HI'], 120, 60, 3.4146696367776015e-144]
['GI', [], 110, 60, 6578.111422913933]
['HG', ['GI'], 150, 50, 3.407466502782822e-144]
['HI', [], 80, 70, 7.203134007310989e-147]
